In [ ]:
!pip install umap-learn
!pip install hdbscan
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 10.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cach

In [ ]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer
#import hdbscan
#import umap.umap_ as umap
from sklearn.cluster import KMeans

In [ ]:
texts  = ["Looking for a software engineer proficient in Python and JavaScript to work on full-stack development with experience in Django and React.", "Seeking a marketing specialist with expertise in SEO, content strategy, and social media management to develop and execute digital marketing campaigns.", "Hiring a financial analyst to perform data-driven analysis, generate financial reports, and assist in strategic decision-making.", "Looking for a machine learning engineer experienced in building models using TensorFlow and PyTorch to work on predictive analytics projects.", "We are looking for a project manager with PMP certification to oversee software development teams and ensure project delivery within deadlines.", "Seeking a UX/UI designer to create engaging, user-friendly interfaces, with strong skills in Figma and Adobe XD.", "Hiring a data scientist with experience in statistical modeling, data mining, and visualization tools like Tableau and Power BI.", "Looking for a content writer to produce high-quality blog posts, articles, and copy for websites, with excellent research and writing skills.", "Seeking an HR manager to handle employee relations, talent acquisition, and performance management for a fast-growing tech startup.", "We are hiring a customer support specialist to manage client queries and provide exceptional service through multiple channels, including chat, phone, and email."]

In [ ]:
summarizer = pipeline("summarization")
summarized_texts = [summarizer(text, max_length=50, min_length=25, do_sample=False)[0]['summary_text'] for text in texts]

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Your max_length is set to 50, but your input_length is only 26. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Your max_length is set to 50, but your input_length is only 26. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Your max_length is set to 50, but your input_length is only 26. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Your max_length i

In [ ]:
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')
sentence_embeddings = model.encode(summarized_texts)

In [ ]:
n_clusters = 5 # Desired number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=0)
cluster_labels = kmeans.fit_predict(sentence_embeddings)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
print("Summarization + Sentence Transformers Cluster labels:", cluster_labels)
def print_clusters(texts, labels, title):
    clusters = {}
    for i, label in enumerate(labels):
        if label not in clusters:
            clusters[label] = []
        clusters[label].append(texts[i])

    print(f"\n{title} Clusters:")
    for label, sentences in clusters.items():
        print(f"Cluster {label}:")
        for sentence in sentences:
            print(f"  {sentence}")

Summarization + Sentence Transformers Cluster labels: [4 2 0 4 3 1 0 2 3 2]


In [ ]:
print_clusters(texts, cluster_labels, 'Summarization + Sentence Transformers')


Summarization + Sentence Transformers Clusters:
Cluster 4:
  Looking for a software engineer proficient in Python and JavaScript to work on full-stack development with experience in Django and React.
  Looking for a machine learning engineer experienced in building models using TensorFlow and PyTorch to work on predictive analytics projects.
Cluster 2:
  Seeking a marketing specialist with expertise in SEO, content strategy, and social media management to develop and execute digital marketing campaigns.
  Looking for a content writer to produce high-quality blog posts, articles, and copy for websites, with excellent research and writing skills.
  We are hiring a customer support specialist to manage client queries and provide exceptional service through multiple channels, including chat, phone, and email.
Cluster 0:
  Hiring a financial analyst to perform data-driven analysis, generate financial reports, and assist in strategic decision-making.
  Hiring a data scientist with experienc

In [ ]:
import numpy as np
from sklearn.metrics import silhouette_score
valid_labels = cluster_labels != -1
if np.sum(valid_labels) > 1:  # Check if there are more than one valid sample
    silhouette_avg = silhouette_score(sentence_embeddings[valid_labels], cluster_labels[valid_labels])
    print(f"Silhouette Score: {silhouette_avg:.3f}")
else:
    print("Silhouette Score: Not applicable (less than 2 valid samples)")

Silhouette Score: 0.081
